In [1]:
!fx workspace create --prefix ~/workspace --template torch_cnn_histology
%cd ~/workspace

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-11-30 15:21:56.347049: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-11-30 15:21:56.347077: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Creating Workspace Directories
Creating Workspace Certifications
Creating Workspace Templates
No additional requirements for workspace defined. Skipping...

New workspace directory structure:
workspace
├── code
│   ├── pthistology_inmemory.py
│   ├── pt_cnn.py
│   ├── histology_utils.py
│   ├── requirements.txt
│   ├── __pycache__
│   │   └── __init__.cpython-36.pyc
│   └── __init__.py
├── agg_to_col_two_signed_cert.zip
├── col_two_to_agg_cert_request.zip
├── plan
│   ├── cols.yaml
│   ├── data.yaml
│   ├── defaults
│   │   

In [2]:
!fx workspace certify

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-11-30 15:21:58.918482: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-11-30 15:21:58.918507: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Setting Up Certificate Authority...

1.  Create Root CA
1.1 Create Directories
1.2 Create Database
1.3 Create CA Request
1.4 Create CA Certificate
2.  Create Signing Certificate
2.1 Create Directories
2.2 Create Database
2.3 Create Signing Certificate CSR
2.4 Sign Signing Certificate CSR
3   Create Certificate Chain

Done.

 ✔️ OK


In [3]:
!fx aggregator generate-cert-request

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-11-30 15:22:01.530700: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-11-30 15:22:01.530726: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Creating AGGREGATOR certificate key pair with following settings: CN=nnlicv611.inn.intel.com, SAN=DNS:nnlicv611.inn.intel.com
  Moving AGGREGATOR certificate key pair to: cert/server

 ✔️ OK


In [4]:
!fx aggregator certify -s 

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-11-30 15:22:03.945595: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-11-30 15:22:03.945617: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
The CSR Hash for file agg_nnlicv611.inn.intel.com.csr =  bb9304c8fbfb38fe8d820929b085f10068f8de5dd9314b47c62745ecf976d3ce

 Signing AGGREGATOR certificate key pair
  Moving AGGREGATOR certificate key pair to: cert/server

 ✔️ OK


In [5]:
!fx collaborator generate-cert-request -n one -d 1

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-11-30 15:22:06.232201: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-11-30 15:22:06.232228: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Creating COLLABORATOR certificate key pair with following settings: CN=one, SAN=DNS:one
  Moving COLLABORATOR certificate to: cert/col_one
Archive col_one_to_agg_cert_request.zip with certificate signing request created
This file should be sent to the certificate authority (typically hosted by the aggregator) for signing

 ✔️ OK


In [6]:
!fx collaborator certify -s -n one

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-11-30 15:22:08.559253: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-11-30 15:22:08.559277: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
The CSR Hash for file col_one.csr =  e242c548adfc9fa850a3cb08c08750528becb8d9549c6f3dde9358fc7f16a433

 Signing COLLABORATOR certificate key pair
  Moving COLLABORATOR certificate key pair to: cert/client

Registering one in plan/cols.yaml

 ✔️ OK


In [7]:
!fx collaborator generate-cert-request -n two -d 2

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-11-30 15:22:10.890762: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-11-30 15:22:10.890787: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Creating COLLABORATOR certificate key pair with following settings: CN=two, SAN=DNS:two
  Moving COLLABORATOR certificate to: cert/col_two
Archive col_two_to_agg_cert_request.zip with certificate signing request created
This file should be sent to the certificate authority (typically hosted by the aggregator) for signing

 ✔️ OK


In [8]:
!fx collaborator certify -s -n two

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-11-30 15:22:13.210791: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-11-30 15:22:13.210817: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
The CSR Hash for file col_two.csr =  a30f17e5301be4897a0c9315b1aa4e93989aeced91e3fd2216706284bbf87f4c

 Signing COLLABORATOR certificate key pair
  Moving COLLABORATOR certificate key pair to: cert/client

Registering two in plan/cols.yaml

 ✔️ OK


In [9]:
!fx plan initialize

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-11-30 15:22:15.559392: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-11-30 15:22:15.559419: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[15:22:16] INFO     Loading DEFAULTS for section aggregator from file plan/defaults/aggregator.yaml.                                                  ]8;id=1606738936.567986-676657;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/federated/plan/plan.py\plan.py]8;;\:87
           INFO     Loading DEFAULTS for section assigner from file plan/defaults/assigner.yaml.                                                      ]8;id=1606738936.5695689-910682;file:///home/itrushkin/.virtualenvs/fledge/li

100%|███████████████████████▉| 257662976/258098431 [03:30<00:00, 4141847.44it/s][15:25:51] INFO     Histology > X_train Shape : (2000, 3, 150, 150)                                                                       ]8;id=1606739151.411712-414395;file:///home/itrushkin/workspace/code/histology_utils.py\histology_utils.py]8;;\:128
           INFO     Histology > y_train Shape : (2000,)                                                                                   ]8;id=1606739151.4131856-849377;file:///home/itrushkin/workspace/code/histology_utils.py\histology_utils.py]8;;\:129
           INFO     Histology > Train Samples : 2000                                                                                      ]8;id=1606739151.4145653-275346;file:///home/itrushkin/workspace/code/histology_utils.py\histology_utils.py]8;;\:130
           INFO     Histology > Valid Samples : 500                                                                                       ]8;id

In [10]:
!fx aggregator start

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-11-30 15:25:55.815979: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-11-30 15:25:55.816004: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[15:25:56] INFO     Loading DEFAULTS for section aggregator from file plan/defaults/aggregator.yaml.                                                  ]8;id=1606739156.8444686-785142;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/federated/plan/plan.py\plan.py]8;;\:87
           INFO     Loading DEFAULTS for section assigner from file plan/defaults/assigner.yaml.                                                      ]8;id=1606739156.846181-292458;file:///home/itrushkin/.virtualenvs/fledge/li

[15:25:57] INFO     Starting Aggregator gRPC Server                                                                                                ]8;id=1606739157.0216036-92048;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/transport/grpc/server.py\server.py]8;;\:145
[15:26:41] INFO     Aggregator GetTasks function reached from collaborator one...                                                              ]8;id=1606739201.1317947-138330;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:244
           INFO     Sending tasks to collaborator one for round 0                                                                              ]8;id=1606739201.1367216-106887;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:278
[15:26:42] INFO     Collaborator one is sending task results for a

[15:27:10] INFO     Collaborator two is sending task results for aggregated_model_validation, round 1                                          ]8;id=1606739230.3368018-513301;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:437
[15:27:15] INFO     Aggregator GetTasks function reached from collaborator one...                                                              ]8;id=1606739235.0208273-697118;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:244
           INFO     Sending tasks to collaborator one for round 1                                                                              ]8;id=1606739235.0255454-457460;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:278
[15:27:17] INFO     Collaborator one is sending task re

[15:27:32] INFO     Collaborator one is sending task results for aggregated_model_validation, round 2                                          ]8;id=1606739252.9561405-509956;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:437
[15:27:33] INFO     Collaborator two is sending task results for aggregated_model_validation, round 2                                          ]8;id=1606739253.8261983-804599;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:437
[15:27:47] INFO     Collaborator one is sending task results for train, round 2                                                                ]8;id=1606739267.2117896-592417;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:437
[15:27:49] INFO     Collaborator two is sending task re

           INFO     Collaborator two is sending task results for train, round 3                                                                ]8;id=1606739285.6849377-335037;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:437
[15:28:06] INFO     Collaborator two is sending task results for locally_tuned_model_validation, round 3                                       ]8;id=1606739286.3977308-859467;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:437
           INFO     Aggregator GetTasks function reached from collaborator two...                                                              ]8;id=1606739286.4041514-851045;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:244
[15:28:15] INFO     Collaborator one is sending task re

...

[15:32:47] INFO     Collaborator two is sending task results for train, round 14                                                               ]8;id=1606739567.4311218-865387;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:437
[15:32:48] INFO     Collaborator two is sending task results for locally_tuned_model_validation, round 14                                      ]8;id=1606739568.188415-1577;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:437
           INFO     train task metrics...                                                                                                      ]8;id=1606739568.1997948-801222;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:605
           INFO     CrossEntropyLoss:       0.7166        

[15:33:13] INFO     locally_tuned_model_validation task metrics...                                                                             ]8;id=1606739593.2924204-302671;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:605
           INFO     acc:    0.7818                                                                                                             ]8;id=1606739593.3021393-286551;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:639
           INFO     aggregated_model_validation task metrics...                                                                                ]8;id=1606739593.3034148-923798;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:605
           INFO     acc:    0.7728                     

[15:33:38] INFO     Saving round 17 model...                                                                                                   ]8;id=1606739618.0299034-772949;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:702
           INFO     Starting round 17...                                                                                                       ]8;id=1606739618.2816925-47111;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:710
           INFO     Aggregator GetTasks function reached from collaborator two...                                                              ]8;id=1606739618.2859924-561299;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:244
           INFO     Sending tasks to collaborator two fo

[15:34:09] INFO     Collaborator one is sending task results for aggregated_model_validation, round 18                                         ]8;id=1606739649.180378-109118;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:437
[15:34:12] INFO     Aggregator GetTasks function reached from collaborator two...                                                              ]8;id=1606739652.4565964-583862;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:244
           INFO     Sending tasks to collaborator two for round 18                                                                             ]8;id=1606739652.4614968-769025;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:278
[15:34:14] INFO     Collaborator two is sending task res

[15:34:33] INFO     Collaborator one is sending task results for aggregated_model_validation, round 19                                         ]8;id=1606739673.1798196-292373;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:437
[15:34:43] INFO     Collaborator two is sending task results for train, round 19                                                               ]8;id=1606739683.1526678-378287;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:437
[15:34:47] INFO     Collaborator one is sending task results for train, round 19                                                               ]8;id=1606739687.2748876-383229;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/aggregator/aggregator.py\aggregator.py]8;;\:437
[15:34:48] INFO     Collaborator two is sending task re